In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
import pickle
import json
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical

class SimpleIntentClassifier:
    def __init__(self, model_type='neural_network'):
        """
        Initialize the classifier
        model_type: 'neural_network' or 'tfidf_mlp'
        """
        self.model_type = model_type
        self.vectorizer = TfidfVectorizer(
            max_features=1000,
            ngram_range=(1, 2),
            stop_words='english'
        )
        self.label_encoder = LabelEncoder()
        self.model = None

    def load_data(self, file_path):
        """Load data from JSON file."""
        with open(file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)

        df = pd.DataFrame(data)
        df['text'] = df['text'].str.strip()
        return df

    def preprocess_data(self, df):
        """Preprocess the data."""
        # Encode labels
        y = self.label_encoder.fit_transform(df['intent'])

        # Convert text to TF-IDF features
        X = self.vectorizer.fit_transform(df['text'])

        if self.model_type == 'neural_network':
            y = to_categorical(y)

        return X, y

    def build_neural_network(self, input_dim, num_classes):
        """Build a simple neural network."""
        model = Sequential([
            Dense(256, activation='relu', input_shape=(input_dim,)),
            Dropout(0.3),
            Dense(128, activation='relu'),
            Dropout(0.2),
            Dense(64, activation='relu'),
            Dropout(0.1),
            Dense(num_classes, activation='softmax')
        ])

        model.compile(
            optimizer=Adam(learning_rate=0.001),
            loss='categorical_crossentropy',
            metrics=['accuracy']
        )

        return model

    def train(self, file_path, test_size=0.2):
        """Train the model."""
        # Load and preprocess data
        df = self.load_data(file_path)
        X, y = self.preprocess_data(df)

        # Split data
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=test_size, random_state=42, stratify=y if self.model_type != 'neural_network' else np.argmax(y, axis=1)
        )

        # Convert sparse matrix to dense for neural network
        if self.model_type == 'neural_network':
            X_train = X_train.toarray()
            X_test = X_test.toarray()

            # Build and train neural network
            self.model = self.build_neural_network(X_train.shape[1], y_train.shape[1])

            history = self.model.fit(
                X_train, y_train,
                epochs=50,
                batch_size=32,
                validation_split=0.2,
                verbose=1
            )

            # Evaluate
            y_pred = self.model.predict(X_test)
            y_pred_classes = np.argmax(y_pred, axis=1)
            y_test_classes = np.argmax(y_test, axis=1)

        print("\nClassification Report:")
        print(classification_report(
            y_test_classes if self.model_type == 'neural_network' else y_test,
            y_pred_classes if self.model_type == 'neural_network' else y_pred,
            target_names=self.label_encoder.classes_
        ))

        return history if self.model_type == 'neural_network' else None

    def predict(self, text):
        """Predict intent for new text."""
        # Vectorize text
        X = self.vectorizer.transform([text])

        if self.model_type == 'neural_network':
            X = X.toarray()
            # Get prediction probabilities
            probs = self.model.predict(X)[0]
            # Get predicted class
            pred_class = np.argmax(probs)
            confidence = float(probs[pred_class])

        # Convert prediction to intent label
        predicted_intent = self.label_encoder.inverse_transform([pred_class])[0]

        return {
            'intent': predicted_intent,
            'confidence': confidence
        }

    def save_model(self, path_prefix):
        """Save the model and preprocessors."""
        if self.model_type == 'neural_network':
            self.model.save(f'{path_prefix}_model.h5')

        # Save vectorizer and label encoder
        with open(f'{path_prefix}_vectorizer.pkl', 'wb') as f:
            pickle.dump(self.vectorizer, f)

        with open(f'{path_prefix}_label_encoder.pkl', 'wb') as f:
            pickle.dump(self.label_encoder, f)

    def load_model(self, path_prefix):
        """Load the model and preprocessors."""
        if self.model_type == 'neural_network':
            self.model = tf.keras.models.load_model(f'{path_prefix}_model.h5')

        # Load vectorizer and label encoder
        with open(f'{path_prefix}_vectorizer.pkl', 'rb') as f:
            self.vectorizer = pickle.load(f)

        with open(f'{path_prefix}_label_encoder.pkl', 'rb') as f:
            self.label_encoder = pickle.load(f)

In [ ]:
def main():
    # Initialize classifier
    classifier = SimpleIntentClassifier(model_type='neural_network')

    # Train model
    history = classifier.train('/content/jakarta_transport_intents_813_20241213_230805.json')

    # Save model
    classifier.save_model('intent_classifier')

    # Test predictions
    test_texts = [
        "guide me to Blok M station",
        "where is the nearest elevator",
        "recommend a good restaurant near Sudirman station"
    ]

    for text in test_texts:
        prediction = classifier.predict(text)
        print(f"\nText: {text}")
        print(f"Predicted Intent: {prediction['intent']}")
        print(f"Confidence: {prediction['confidence']:.4f}")

In [ ]:
main()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
17/17 ━━━━━━━━━━━━━━━━━━━━ 8s 217ms/step - accuracy: 0.4365 - loss: 1.0883 - val_accuracy: 0.6692 - val_loss: 1.0055
Epoch 2/50
17/17 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.6828 - loss: 0.9467 - val_accuracy: 0.7923 - val_loss: 0.6486
Epoch 3/50
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8701 - loss: 0.5344 - val_accuracy: 0.9308 - val_loss: 0.2539
Epoch 4/50
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9813 - loss: 0.1304 - val_accuracy: 0.9538 - val_loss: 0.1812
Epoch 5/50
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9839 - loss: 0.0461 - val_accuracy: 0.9385 - val_loss: 0.1473
Epoch 6/50
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9935 - loss: 0.0253 - val_accuracy: 0.9692 - val_loss: 0.1426
Epoch 7/50
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9967 - loss: 0.0120 - val_accuracy: 0.9462 - val_loss: 0.1412
Epoch 8/50
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 1.0000 - loss: 0.0064 - val_accuracy: 0.9615 - val_los


Classification Report:
                        precision    recall  f1-score   support

analyzing_surroundings       0.97      1.00      0.98        65
  asking_for_direction       0.96      0.91      0.93        53
service_recommendation       0.91      0.93      0.92        45

              accuracy                           0.95       163
             macro avg       0.95      0.95      0.95       163
          weighted avg       0.95      0.95      0.95       163

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 164ms/step

Text: guide me to Blok M station
Predicted Intent: asking_for_direction
Confidence: 0.9745
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step

Text: where is the nearest elevator
Predicted Intent: analyzing_surroundings
Confidence: 0.9162
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step

Text: recommend a good restaurant near Sudirman station
Predicted Intent: analyzing_surroundings
Confidence: 0.9564


In [ ]:
import tensorflow as tf

In [ ]:
classifier = SimpleIntentClassifier(model_type='neural_network')
classifier.load_model('intent_classifier')

In [ ]:
# Test predictions
test_texts = [
    "guide me to Blok M station",
    "where is the nearest elevator",
    "recommend a good restaurant near Sudirman station"
]

for text in test_texts:
    prediction = classifier.predict(text)
    print(f"\nText: {text}")
    print(f"Predicted Intent: {prediction['intent']}")
    print(f"Confidence: {prediction['confidence']:.4f}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 400ms/step

Text: guide me to Blok M station
Predicted Intent: asking_for_direction
Confidence: 0.9745
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step

Text: where is the nearest elevator
Predicted Intent: analyzing_surroundings
Confidence: 0.9162
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step

Text: recommend a good restaurant near Sudirman station
Predicted Intent: analyzing_surroundings
Confidence: 0.9564


In [ ]:
# Define comprehensive test cases
test_texts = [
    # Asking for Directions
    "how do I get to Grand Indonesia from Blok M?",
    "which train goes to Sudirman station?",
    "what's the fastest route to Monas from Lebak Bulus?",
    "guide me to Plaza Indonesia using MRT",
    "directions to Kota Tua from Bundaran HI",
    "what's the best way to reach Thamrin City?",

    # Analyzing Surroundings
    "where is the nearest elevator at Dukuh Atas station?",
    "is there a wheelchair ramp at Blok M station?",
    "where can I find the ticket machines?",
    "which exit leads to Plaza Indonesia?",
    "are there any obstacles near the escalator?",
    "where is the nearest emergency exit?",

    # Service Recommendations
    "what's the best time to avoid crowds at Bundaran HI station?",
    "recommend the quickest route to reach SCBD during rush hour",
    "which MRT line should I take to reach Monas?",
    "is it better to take the MRT or TransJakarta to Kuningan?",
    "what's the most convenient way to get to Central Park mall?",
    "suggest a route to avoid traffic to Kemang",

    # Mixed/Complex Queries
    "is there an elevator near exit A at Senayan station?",
    "how do I get to Pacific Place and where's the nearest exit?",
    "what's the fastest route to Tanjung Priok and are there escalators?",
    "recommend a route to Grand Indonesia with minimal walking",
    "guide me to the nearest halte TransJakarta from Sudirman station",
    "which exit at Bundaran HI has the least obstacles for wheelchairs?"
]

# Function to test and analyze predictions
def test_intent_classifier(classifier, texts):
    results = []

    print("Intent Classification Results:")
    print("-" * 80)

    for text in test_texts:
        prediction = classifier.predict(text)
        results.append({
            'text': text,
            'intent': prediction['intent'],
            'confidence': prediction['confidence']
        })

        print(f"\nInput: {text}")
        print(f"Predicted Intent: {prediction['intent']}")
        print(f"Confidence: {prediction['confidence']:.4f}")

    # Analysis of results
    print("\n" + "=" * 80)
    print("Classification Summary:")
    print("=" * 80)

    # Count predictions by intent
    intent_counts = {}
    avg_confidence_by_intent = {}

    for result in results:
        intent = result['intent']
        confidence = result['confidence']

        intent_counts[intent] = intent_counts.get(intent, 0) + 1

        if intent in avg_confidence_by_intent:
            avg_confidence_by_intent[intent].append(confidence)
        else:
            avg_confidence_by_intent[intent] = [confidence]

    # Print summary statistics
    print("\nPredictions by Intent:")
    for intent, count in intent_counts.items():
        avg_conf = sum(avg_confidence_by_intent[intent]) / len(avg_confidence_by_intent[intent])
        print(f"\n{intent}:")
        print(f"  Count: {count}")
        print(f"  Average Confidence: {avg_conf:.4f}")

    # Find highest and lowest confidence predictions
    sorted_results = sorted(results, key=lambda x: x['confidence'])

    print("\nLowest Confidence Predictions:")
    for result in sorted_results[:3]:
        print(f"  Text: {result['text']}")
        print(f"  Intent: {result['intent']}")
        print(f"  Confidence: {result['confidence']:.4f}\n")

    print("Highest Confidence Predictions:")
    for result in sorted_results[-3:]:
        print(f"  Text: {result['text']}")
        print(f"  Intent: {result['intent']}")
        print(f"  Confidence: {result['confidence']:.4f}\n")

# Usage
if __name__ == "__main__":
    # Initialize your classifier
    classifier = SimpleIntentClassifier()
    classifier.load_model('intent_classifier')

    # Run tests
    test_intent_classifier(classifier, test_texts)

Intent Classification Results:
--------------------------------------------------------------------------------
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 260ms/step

Input: how do I get to Grand Indonesia from Blok M?
Predicted Intent: service_recommendation
Confidence: 0.9172
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step

Input: which train goes to Sudirman station?
Predicted Intent: asking_for_direction
Confidence: 0.9753
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step

Input: what's the fastest route to Monas from Lebak Bulus?
Predicted Intent: asking_for_direction
Confidence: 0.9991
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step

Input: guide me to Plaza Indonesia using MRT
Predicted Intent: asking_for_direction
Confidence: 0.9966
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step

Input: directions to Kota Tua from Bundaran HI
Predicted Intent: asking_for_direction
Confidence: 0.9290
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step

Input: what's the best way to reach Thamrin City?
Predicted Intent: asking_for_direction
Confidence: 0.9997
1/1 ━